In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

In [ ]:
%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 14, 8

RANDOM_SEED = 42

In [ ]:
df = pd.read_csv("D:\dataset\CIC-IDS-2017\MachineLearningCVE\combISCX.csv")

In [ ]:
df["Flow Bytes/s"] = df["Flow Bytes/s"].fillna(df["Flow Bytes/s"].mean())

In [ ]:
df["Flow Bytes/s"].isnull().sum()

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.isnull().values.any()

In [ ]:
len(df.columns)

In [ ]:
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = int((nCol + nGraphPerRow - 1) / nGraphPerRow)
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()
    
    
plotPerColumnDistribution(df, 79, 5)

In [ ]:
def find_outliers(df):
    outliers_columns = []
    for column in df.columns:
        if df[column].dtype != 'object':
            std = df[column].std()
            if std > 0:
                z_score = (df[column] - df[column].mean()) / std
                if (np.abs(z_score) > 3).any():
                    outliers_columns.append(column)
        return outliers_columns

outliers_columns = find_outliers(df)
print("Columns with outliers (3 standard deviations or more away from the mean):", outliers_columns)

In [ ]:
def find_invalid_columns(df):
    invalid_columns = []
    for column in df.columns:
        if df[column].dtype != object:  # 문자열이 아닌 경우에만 확인합니다.
            if np.any(np.isinf(df[column])) or np.any(np.abs(df[column]) > 1e10 ):
                invalid_columns.append(column)
    return invalid_columns

# 데이터프레임 df에서 불가능한 값을 포함하는 컬럼 찾기
invalid_cols = find_invalid_columns(df)

if invalid_cols:
    print("The following columns contain invalid values:")
    print(invalid_cols)
else:
    print("No columns contain invalid values.")

In [ ]:
#tmp = df[" Destination Port"]
#df = df.drop(columns=" Destination Port")

In [ ]:
sorted_df = df.sort_values(by='Flow Bytes/s', ascending=False)
print(sorted_df['Flow Bytes/s'])

In [ ]:
threshold = 1e8  # 예시 임계값
inf_col = ['Flow Bytes/s', ' Flow Packets/s', ' Fwd Header Length', ' Fwd Header Length.1']

for col in inf_col:
    df[col] = np.clip(df[col], -threshold, threshold)

In [ ]:

#Split dataset on train and test
from sklearn.model_selection import train_test_split
train, test=train_test_split(df,test_size=0.3, random_state=10)

#Exploratory Analysis
# Descriptive statistics
train.describe()
test.describe()

# Packet Attack Distribution
train[' Label'].value_counts()
test[' Label'].value_counts()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


# extract numerical attributes and scale it to have zero mean and unit variance  
cols = train.select_dtypes(include=['float64','int64']).columns
sc_train = scaler.fit_transform(train.select_dtypes(include=['float64','int64']))
sc_test = scaler.fit_transform(test.select_dtypes(include=['float64','int64']))

# turn the result back to a dataframe
sc_traindf = pd.DataFrame(sc_train, columns = cols)
sc_testdf = pd.DataFrame(sc_test, columns = cols)

In [ ]:
# importing one hot encoder from sklearn 
from sklearn.preprocessing import OneHotEncoder 

# creating one hot encoder object 
onehotencoder = OneHotEncoder() 

trainDep = train[' Label'].values.reshape(-1,1)
trainDep = onehotencoder.fit_transform(trainDep).toarray()
testDep = test[' Label'].values.reshape(-1,1)
testDep = onehotencoder.fit_transform(testDep).toarray()

In [ ]:
train_X=sc_traindf
train_y=trainDep[:,0]

test_X=sc_testdf
test_y=testDep[:,0]

In [ ]:
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler

# Assuming train_X and test_X are your feature matrices, and train_y and test_y are your target vectors

# Standardize the data (important before applying PCA)
scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_X)
test_X_scaled = scaler.transform(test_X)

# Apply PCA
num_components = 30  # Use 10 components for PCA
pca = PCA(n_components=num_components)

# Fit and transform the training data
train_X_pca = pca.fit_transform(train_X_scaled)

# Transform the testing data (using the same PCA transformation)
test_X_pca = pca.transform(test_X_scaled)

# Select top 20 features using SelectKBest with the f_classif scoring function
feature_selector = SelectKBest(score_func=f_classif, k=20)  # Select top 20 features

# Fit and transform the training data
train_X_selected = feature_selector.fit_transform(train_X_pca, train_y)

# Transform the testing data (using the same feature selection)
test_X_selected = feature_selector.transform(test_X_pca)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D

# Assuming train_X_selected and test_X_selected are your selected features after PCA and feature selection

# Specify the number of selected features
num_selected_features = train_X_selected.shape[1]

# Reshape the data for 1D CNN (assuming you have time series-like data)
train_X_selected = train_X_selected.reshape(train_X_selected.shape[0], num_selected_features, 1)
test_X_selected = test_X_selected.reshape(test_X_selected.shape[0], num_selected_features, 1)

# Create a simple CNN model
model = Sequential()

# Convolutional layers
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(num_selected_features, 1)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(num_selected_features, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Flatten layer to connect convolutional layers to dense layers
model.add(Flatten())

# Dense layers
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

# Additional hidden layers
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))

model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification, adjust for your task

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_X_selected, train_y, epochs=5, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_X_selected, test_y)
print(f'Test Accuracy: {test_accuracy}')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Predict on test set
predictions = model.predict(test_X_selected)
predictions = (predictions > 0.5)

conf_matrix = confusion_matrix(test_y, predictions)
TN = conf_matrix[0, 0]  # True Negative
FP = conf_matrix[0, 1]  # False Positive
TP = conf_matrix[1, 1]  # True Positive
FN = conf_matrix[1, 0]  # False Negative

# Calculate specificity
specificity = TN / (TN + FP)


# Calculate additional evaluation metrics
print("Confusion Matrix:")
print(confusion_matrix(test_y, predictions))

print("\nClassification Report:")
print(classification_report(test_y, predictions))

print("\nSpecificity:")
print(specificity)

In [ ]:
from sklearn.metrics import classification_report

# 기존 코드
report_dict = classification_report(test_y, predictions, output_dict=True)

# 소수점 5자리까지 출력
print("Classification Report (소수점 5자리):")
for key, value in report_dict.items():
    if isinstance(value, dict):
        for sub_key, sub_value in value.items():
            print(f"{key} {sub_key}: {sub_value:.5f}")
    else:
        print(f"{key}: {value:.5f}")